
# OPTIMAL V3 THRESHOLD CALCULATOR

## Run ideally once a week.

<br/><br/>

![](https://gifdb.com/images/high/the-more-you-know-dog-meme-a60n5320bzqqt6gb.webp)

#### We first set the path to where the initial data is located on the pc and then we load the data into a dataframe for bteasier manipulation.

#### The data is provided by Cox


In [19]:

# update PATH with location where the file is on your machine
FILEPATH = '/Users/jonathankhalifa/Desktop/ITERATIONS/v3_data.xlsx'
df = pd.read_excel(FILEPATH, index_col=0)
df

,O,H,L,C,prob_open,prob_close
date,,,,,,
2021-08-04,4415.950195,4416.169922,4400.229980,4402.660156,0.77,0.79
2021-08-05,4408.859863,4429.759766,4408.859863,4429.100098,0.69,0.67
2021-08-06,4429.069824,4440.819824,4429.069824,4436.520020,0.55,0.51
2021-08-09,4437.770020,4439.390137,4424.740234,4432.350098,0.53,0.50
2021-08-10,4435.790039,4445.209961,4430.029785,4436.750000,0.54,0.49
...,...,...,...,...,...,...
2022-08-05,4115.870117,4151.580078,4107.310059,4145.189941,0.89,0.88
2022-08-08,4155.930176,4186.620117,4128.970215,4140.060059,0.90,0.94
2022-08-09,4133.109863,4137.299805,4112.089844,4122.470215,0.90,0.95


<br/><br/>
# Generate all thresholds OnOffs and Perfs

#### From the initial O H L C table with v3 model's probas we calc the OnOffs and daily performance for all the different threshold combinations.
- The range from OFF to ON is 50 - 90.
- The range from ON to OFF is 49 - 10.
##### example : 73-41 means that in order to switch from off to on , we need the proba to be higher or equal to 73. Then from on to off it has to be below of equal to 41.

#### The final results are then exported to "TH_PERFS_ALL.xlsx".

In [30]:
import pandas as pd
from itertools import product


def calc_on_off(proba):
    global prev, on_off, off_on
    threshold = on_off if (prev == 'On') else off_on
    prev = 'On' if proba >= threshold else 'Off'
    return prev


default_state = 'Off'
proba = 100*df.prob_open
r_on_off = range(10, 50)
r_off_on = range(50, 90)
thresholdColumnTitlesArray = []
perfColumnTitlesArray = []


# create all on off columns
for on_off, off_on in product(r_on_off, r_off_on):
    prev = default_state
    df[f'{off_on}-{on_off}'] = proba.apply(calc_on_off)
    df[f'perf {off_on}-{on_off}'] = 0
    thresholdColumnTitlesArray.append(f'{off_on}-{on_off}')
    perfColumnTitlesArray.append(f'perf {off_on}-{on_off}')
    
    
# ensure numeric values
df['O'] = pd.to_numeric(df['O'])
df['C'] = pd.to_numeric(df['C'])

def calcPerf(s):
    m1 = s.eq('On')
    m2 = s.eq('On').shift()
    m3 = s.eq('Off').shift()

    return np.select([m1&m2, m1&m3, m2&~m1],
                     [df['C'].div(df['C'].shift()).sub(1).mul(100),
                      df['O'].div(df['C'].where(m1).ffill()).sub(1).mul(100),
                      df['O'].div(df['C'].shift()).sub(1).mul(100)
                      ], 0
                      )

# apply to each column and update
df.update(df[thresholdColumnTitlesArray].apply(calcPerf).add_prefix('perf '))

# extract to excel
df.to_excel("TH_PERFS_ALL.xlsx")
df

,O,H,L,C,prob_open,prob_close,50-10,perf 50-10,51-10,perf 51-10,...,85-49,perf 85-49,86-49,perf 86-49,87-49,perf 87-49,88-49,perf 88-49,89-49,perf 89-49
date,,,,,,,,,,,,,,,,,,,,,
2021-08-04,4415.950195,4416.169922,4400.229980,4402.660156,0.77,0.79,On,0.000000,On,0.000000,...,Off,0.000000,Off,0.000000,Off,0.000000,Off,0.000000,Off,0.000000
2021-08-05,4408.859863,4429.759766,4408.859863,4429.100098,0.69,0.67,On,0.600545,On,0.600545,...,Off,0.000000,Off,0.000000,Off,0.000000,Off,0.000000,Off,0.000000
2021-08-06,4429.069824,4440.819824,4429.069824,4436.520020,0.55,0.51,On,0.167527,On,0.167527,...,Off,0.000000,Off,0.000000,Off,0.000000,Off,0.000000,Off,0.000000
2021-08-09,4437.770020,4439.390137,4424.740234,4432.350098,0.53,0.50,On,-0.093991,On,-0.093991,...,Off,0.000000,Off,0.000000,Off,0.000000,Off,0.000000,Off,0.000000
2021-08-10,4435.790039,4445.209961,4430.029785,4436.750000,0.54,0.49,On,0.099268,On,0.099268,...,Off,0.000000,Off,0.000000,Off,0.000000,Off,0.000000,Off,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-05,4115.870117,4151.580078,4107.310059,4145.189941,0.89,0.88,On,-0.162575,On,-0.162575,...,On,-0.162575,On,-0.162575,On,-0.162575,On,-0.162575,On,-0.162575
2022-08-08,4155.930176,4186.620117,4128.970215,4140.060059,0.90,0.94,On,-0.123755,On,-0.123755,...,On,-0.123755,On,-0.123755,On,-0.123755,On,-0.123755,On,-0.123755
2022-08-09,4133.109863,4137.299805,4112.089844,4122.470215,0.90,0.95,On,-0.424869,On,-0.424869,...,On,-0.424869,On,-0.424869,On,-0.424869,On,-0.424869,On,-0.424869


<br/><br/>
# Calc total performances

#### We sum each perf column and sort the result in order to determin the best performing thresholds. 
#### The final results are then exported to "RESULTS_SORTED.xlsx".


In [31]:
perf_results_df = df[df.columns.intersection(perfColumnTitlesArray)]
perf_results_df = perf_results_df.sum(axis = 0)
perf_results_df_sort = perf_results_df.sort_values(ascending=False)
perf_results_df_sort.to_excel("RESULTS_SORTED.xlsx")
perf_results_df_sort

perf 78-33    12.775202
perf 78-31    12.775202
perf 78-29    12.775202
perf 78-32    12.775202
perf 78-30    12.775202
                ...    
perf 51-38   -33.269425
perf 50-38   -33.269425
perf 52-37   -33.269425
perf 51-37   -33.269425
perf 50-37   -33.269425
Length: 1600, dtype: float64